<a href="https://colab.research.google.com/github/HlyamHtetKyaw/pl-journey/blob/main/PL_XOR_MLP_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-lightning

In [2]:
import pytorch_lightning as pl
import torch
from torch import nn, optim
from torch.autograd import Variable
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader

In [3]:
print("torch vision: ",torch.__version__)
print("pytorch lightening version: ",pl.__version__)

torch vision:  2.6.0+cu124
pytorch lightening version:  2.5.2


In [4]:
# Xor inputs
xor_inputs = [Variable(torch.Tensor([0,0])),
              Variable(torch.Tensor([0,1])),
              Variable(torch.Tensor([1,0])),
              Variable(torch.Tensor([1,1]))];

In [5]:
# Target outputs
xor_targets = [Variable(torch.Tensor([0])),
               Variable(torch.Tensor([1])),
               Variable(torch.Tensor([1])),
               Variable(torch.Tensor([0]))];

In [6]:
# Making data set for input and target, also loading data into train_loader
xor_data = list(zip(xor_inputs,xor_targets));
train_loader = DataLoader(xor_data,batch_size=1);

In [11]:
# Initializing XORModel
class XORModel(pl.LightningModule):
  def __init__(self):
    super(XORModel,self).__init__();
    self.input_layer = nn.Linear(2,4);
    self.output_layer = nn.Linear(4,1);
    self.sigmoid = nn.Sigmoid();
    self.loss = nn.MSELoss();

  def forward(self,input):
    # print("INPUT: ",input.shape);
    x = self.input_layer(input);
    # print("FIRST: ",x.shape);
    x = self.sigmoid(x);
    # print("SECOND: ",x.shape);
    output = self.output_layer(x)
    # print("THIRD: ",output.shape);
    return output;

  def configure_optimizers(self):
    params = self.parameters();
    optimizer = optim.Adam(params=params,lr=0.01);
    return optimizer;

  def training_step(self,batch,batch_idx):
    xor_input, xor_target = batch;
    # print("XOR INPUT: ",xor_input.shape);
    # print("XOR TARGET: ",xor_target);
    outputs = self(xor_input);
    # print("XOR OUTPUT: ",outputs.target);
    loss = self.loss(outputs,xor_target);
    return loss;

In [10]:
# Configuring trainer
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS;
checkpoint_callback = ModelCheckpoint();
model = XORModel();

trainer = pl.Trainer(max_epochs = 100,callbacks=[checkpoint_callback]);

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [12]:
# Training data
trainer.fit(model,train_dataloaders=train_loader);

INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type    | Params | Mode 
-------------------------------------------------
0 | input_layer  | Linear  | 12     | train
1 | output_layer | Linear  | 5      | train
2 | sigmoid      | Sigmoid | 0      | train
3 | loss         | MSELoss | 0      | train
-------------------------------------------------
17        Trainable params
0         Non-trainable params
17        Total params
0.000     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


In [15]:
# Displaying trained model with version
%ls lightning_logs/

version_0/
